In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os

Class object is defined by the author for each subsystem of the model. This simplifies the task of assigning and manipulating the data. 

__Class : Cell__

In [2]:
class cell:
    
    def __init__(self,capacity, weight, vmax, vnom, vmin):
        
        # Cell capacity in Amp-hr
        self.capacity = capacity
        
        # Cell weight in g
        self.weight = weight
        
        
        self.vmax = vmax
        self.vnom = vnom
        self.vmin = vmin
        
    # Defining methods for cell class
    
    def energy(self):
        
        # Cell energy in Watt-hr
        return self.vnom*self.capacity
    
    def specific_energy(self):
        
        # Cell specific energy Wh/kg
        return (1000*self.capacity*self.vnom/self.weight)

__Instancing the Cell class__

In [16]:
cell1 = cell(15,450,4.2,3.8,3.0)

print(cell1.energy())
print(cell1.specific_energy())

57.0
126.66666666666667


__Class : Module__

In [4]:
class module(cell):
    
    def __init__(self, numpar, numser, overhead, class_cell):
        
        # Importing the attributes of the Class: Cell
        self.cell_cap = class_cell.capacity
        self.cell_weight = class_cell.weight
        self.cell_energy = class_cell.energy()
        
        # Number of parallel cells
        self.numpar = numpar
        
        # Number of series cells
        self.numser = numser
        
        # Overhead percentage
        # e.g 10% overhead for packaging, electonics, cooling etc
        self.overhead = overhead/100
        
        # Parameters of the cell
        self.cell = cell
    
    # Defining methods for module class

    def numcell(self):
        
        return (self.numpar*self.numser)
    
    def mod_cap(self):
        
        return (self.numpar*self.cell_cap)
    
    def mod_weight(self):
        
        # Module weight in Kg
        return (self.numcell()*self.cell_weight*(1/(1-self.overhead))/1000)
    
    def mod_energy(self):
        
        # Module energy in KWh
        return (self.numcell()*self.cell_energy/1000)
    
    def mod_spec_energy(self):
        
        # Module specific energy in Wh/kg
        return (1000*self.mod_energy()/self.mod_weight())
        

__Instancing the Module class__

In [17]:
mod = module(3,8,8,cell1)

print(mod.mod_cap())
print(mod.mod_weight())
print(mod.mod_energy())
print(mod.mod_spec_energy())

45
11.739130434782608
1.368
116.53333333333335


__Class : Pack__

In [6]:
class pack(module):
    
    def __init__(self,num_mod_ser,mod_overhead,pack_soc_full,pack_soc_empty,eff,class_cell,class_module):
        
        # Importing the attributes of the Class : Module
        self.mod_numcell = class_module.numcell()
        self.mod_weight = class_module.mod_weight()
        self.mod_energy = class_module.mod_energy()
        self.mod_ser = class_module.numser
        
        # Importing the attributes of the Class : Cell
        self.cell_vmax = class_cell.vmax
        self.cell_vnom = class_cell.vnom
        self.cell_vmin = class_cell.vmin
        
  
        # Number of modules connected in series
        self.num_mod = num_mod_ser
        
        # Pack assembly overheads
        self.overhead = mod_overhead/100
        
        # Full design SOC of the pack
        self.soc_full = pack_soc_full
        
        # Empty design SOC of the pack
        self.soc_empty = pack_soc_empty
        
        # Energy efficiency of the pack
        self.eff = eff/100
        
    # Defing methods for pack class
        
    def weight(self):
            
            return (self.mod_weight*self.num_mod*(1/(1-self.overhead)))
        
    def energy(self):
            
            # Pack energy in KWh
            return (self.mod_energy*self.num_mod)
        
    def specific_energy(self):
            
            # Pack specific energy in Wh/kg
            return (1000*self.energy()/self.weight())
        
    def vmax(self):
            
            # Max voltage of the pack
            return (self.num_mod*self.mod_ser*self.cell_vmax)
        
    def vnom(self):
            
            # Nominal voltage of the pack
            return (self.num_mod*self.mod_ser*self.cell_vnom)
        
    def vmin(self):
            
            # Nominal voltage of the pack
            return (self.num_mod*self.mod_ser*self.cell_vmin)
        
      

__Instancing Pack class__

In [18]:
p1 = pack(12,10,75,25,96,cell1,mod)

print(p1.weight())
print(p1.energy())
print(p1.specific_energy())
print(p1.vmax())
print(p1.vmin())
print(p1.vnom())

156.52173913043475
16.416
104.88000000000002
403.20000000000005
288.0
364.79999999999995


__Class : Motor__

In [8]:
class motor():
    
    def __init__(self,Tmax,RPMrated,RPMmax,eff,inertia):
        
        # Maximum motor torque
        self.Tmax = Tmax
        
        # Rated RPM
        self.rated_RPM = RPMrated
        
        # Maximum RPM
        self.max_RPM = RPMmax
        
        # Motor efficiency
        self.eff = eff/100
        
        # Motor Inertia
        self.inertia = inertia
        
    
    # Defining methods for motor class
    
    def max_power(self):
        
        # Motor power in KW
        return (2*np.pi*self.Tmax*self.rated_RPM/60000)

__Instancing motor class__

In [19]:
m1 = motor(275,4000,12000,95,0.2)

print(m1.max_power())

115.19173063162575


__Class : Wheel__

In [10]:
class wheel():
    
    def __init__(self,radius,inertia,rollcoef):
        
        # Wheel radius in m
        self.radius = radius
        
        # Wheel inertia in kg-m2
        self.inertia = inertia
        
        # Wheel Rolling Coeffcient
        self.roll_coef = rollcoef

__Instancing wheel class__

In [20]:
w1 = wheel(0.35,8,0.0111)
w1.inertia

8

__Class : Drivetrain__

In [12]:
class drivetrain(pack,motor):
    
    def __init__(self,inv_eff,regen_T,gear_ratio,gear_inertia,gear_eff,class_pack,class_motor):
        
        # Importing attributes of Class : Pack
        self.pack_eff = class_pack.eff
        
        # Importing attributes of Class : Motor
        self.motor_eff = class_motor.eff
        
        # Inverter efficiency
        self.inv_eff = inv_eff/100
        
        # Regeneration Torque
        self.T_regen = regen_T
        
        # Gear Ratio
        self.gear_ratio = gear_ratio
        
        # Gear Inerta
        self.gear_inertia = gear_inertia
        
        # Gear Efficiency
        self.gear_eff = gear_eff/100
        
        
    # Defining methods for drivetrain class
    
    def eff(self):
        
#         return(self.pack_eff)
        
        return (self.pack_eff*self.inv_eff*self.motor_eff*self.gear_eff)
          

__Instancing drivetrain class__

In [21]:
d1 = drivetrain(94,0.9,12,0.05,97,p1,m1)

d1.gear_eff
d1.inv_eff


print(d1.eff())

0.8315615999999998


__Class : Vehicle__

In [14]:
class vehicle(wheel,drivetrain):
    
    def __init__(self,num_wheel,road_force,cd,A,weight,payload,Poverhead,class_pack,class_motor,class_wheel,class_drivetrain):
        
        # Importing attributes of Class : Pack
        self.pack_weight = class_pack.weight()
        
        # Importing attributes of CLass : Motor
        self.motor_inertia = class_motor.inertia
        self.motor_max_RPM = class_motor.max_RPM
        
        # Importing attributes of Class : Wheel
        self.wheel_inertia = class_wheel.inertia
        self.wheel_radius = class_wheel.radius
        
        # Importing attribues of Class : DriveTrain
        self.gear_ratio = class_drivetrain.gear_ratio
        self.gear_inertia = class_drivetrain.gear_inertia
        
        
        # Number of wheels
        self.num_wheel = num_wheel
        
        # Road force
        self.road_force = road_force
        
        # Aerodynamic drag coeff
        self.cd = cd
        
        # Vehicle frontal area
        self.farea = A
        
        # Vehicle weight
        self.weight = weight
        
        # Vehicle payload
        self.payload = payload
        
        # Overhead Power
        self.p_overhead = Poverhead
        
    
    # Defining the methods for vehicle class
    
    def curb_weight(self):
        
        # Vehicle curb weight
        return (self.weight + self.pack_weight)
    
    def max_weight(self):
        
        return (self.curb_weight()+self.payload)
    
    def rot_weight(self):
        
        dum = self.motor_inertia + self.gear_inertia
        dum = dum*(self.gear_ratio**2)
        dum = dum + self.wheel_inertia*self.num_wheel
        dum = dum/(self.wheel_radius**2)
        
        return dum
    
    def eq_mass(self):
        
        return (self.max_weight() + self.rot_weight())
    
    def max_speed(self):
        
        # velocity in kmph
        
        dum = 2*np.pi*self.wheel_radius
        dum = dum*self.motor_max_RPM
        dum = dum*60/(1000*self.gear_ratio)
        
        return dum   

__Instancing vehicle class__

In [22]:
v1 = vehicle(4,0,0.22,1.84,1425,75,200,p1,m1,w1,d1)

print(v1.curb_weight())
print(v1.max_weight())
print(v1.rot_weight())
print(v1.eq_mass())
print(v1.max_speed())

1581.5217391304348
1656.5217391304348
555.1020408163266
2211.6237799467613
131.94689145077132
